In [8]:
import pandas as pd
import os
from datetime import datetime, timedelta
import talib
import pandas as pd
import numpy as np
import cufflinks as cf
from plotly.offline import init_notebook_mode, iplot
import plotly.express as px
import plotly.graph_objs as go
import warnings
import enum
warnings.filterwarnings('ignore')
init_notebook_mode(connected=True) 
import sqlite3
import ipywidgets as widgets
from IPython import display
from IPython.display import clear_output
from tabulate import tabulate

In [9]:
class PricePattern(enum.Enum):
    BullishHammer = 1
    BullishHarami = 2
    BullishInverseHammer = 3
    BullishMorningStar = 4
    BullishPiercingLine = 5
    BullishThreeWhiteSoldiers = 6
    BullishEngulfing = 7 

In [10]:
tradingList = {
    "HINDPETRO": {"price": 262.25, "leverage": 10},
    "HINDZINC": {"price": 317.10, "leverage": 10},
    "SBIN": {"price": 364.65, "leverage": 9},
    "ICICIBANK": {"price": 594.75, "leverage": 9},
    "WIPRO": {"price": 507.60, "leverage": 9},
    "ITC": {"price": 203.25, "leverage": 9},
    "BHARTIARTL": {"price": 561.75, "leverage": 9},
    "IOC": {"price": 102.80, "leverage": 9},
    "ONGC": {"price": 115.10, "leverage": 9},
    "DABUR": {"price": 529.20, "leverage": 9},
    "GAIL": {"price": 161.05, "leverage": 9},
    "TATACONSUM": {"price": 640.05, "leverage": 9},
    "AMBUJACEM": {"price": 309.05, "leverage": 9},
    "DLF": {"price": 265.30, "leverage": 8},
    "TATAPOWER": {"price": 109.35, "leverage": 8},
    "TATAMOTORS": {"price": 315.55, "leverage": 7}
}

tickerList = list(tradingList.keys())

hammer_db = "E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\db\\hammer\\statistics.db"
hammer_directory = "E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\test_data\\hammer\\"

bullish_engulfing_db = "E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\db\\bullish_engulfing\\statistics.db"
bullish_engulfing_directory = "E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\test_data\\bullish_engulfing\\"

bullish_harami_db = "E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\db\\bullish_harami\\statistics.db"
bullish_harami_directory = "E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\test_data\\bullish_harami\\"

inverse_hammer_db = "E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\db\\inverse_hammer\\statistics.db"
inverse_hammer_directory = "E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\test_data\\inverse_hammer\\"

morning_star_db = "E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\db\\morning_star\\statistics.db"
morning_star_directory = "E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\test_data\\morning_star\\"

piercing_line_db = "E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\db\\piercing_line\\statistics.db"
piercing_line_directory = "E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\test_data\\piercing_line\\"

white_soldiers_db = "E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\db\\white_soldiers\\statistics.db"
white_soldiers_directory = "E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\test_data\\white_soldiers\\"




db_list = [hammer_db,           
           bullish_harami_db,
           inverse_hammer_db,
           morning_star_db,
           piercing_line_db,
           white_soldiers_db,
           bullish_engulfing_db
          ]
data_directory_list=[hammer_directory,                    
                     bullish_harami_directory,
                     inverse_hammer_directory,
                     morning_star_directory,
                     piercing_line_directory,
                     white_soldiers_directory,
                     bullish_engulfing_db
                    ]
pattern_list=[PricePattern.BullishHammer,              
              PricePattern.BullishHarami,
              PricePattern.BullishInverseHammer,
              PricePattern.BullishMorningStar,
              PricePattern.BullishPiercingLine,
               PricePattern.BullishThreeWhiteSoldiers,
              PricePattern.BullishEngulfing
             ]

bullish_map = {}



In [31]:
class StatisticsManager:

    def __init__(self,name):    
        self.name =name  
        self.db = sqlite3.connect(self.name)
        

    def connectDB(self):
        self.db = sqlite3.connect(self.name)
        
    def get_trade_all(self):
        self.connectDB()
        query = '''SELECT * FROM Statistics'''        
        data = pd.read_sql(query, con=self.db)
        self.close()
        return data
    
    def get_trade_day(self, day):
        self.connectDB()
        query = '''SELECT close_time,current,close,number_of_shares,symbol,type,sum(case when  status = 'loss' then 1 else 0 end) as loss,
                       sum(case when status = 'profit' then 1 else 0 end) as profit,
                       sum(case when status = 'open' then 1 else 0 end) as open,
                       round(sum(case when  status = 'loss' then 1.0 else 0.0 end)/count(*),2) as loss_ratio,
                       round(sum(case when status = 'profit' then 1.0 else 0.0 end)/count(*),2)  as profit_ratio,
                       round(sum(close*number_of_shares -current*number_of_shares ),2)  as returns
                    FROM Statistics GROUP by symbol,close_time having close_time = '''
        query += "'" + day + "'"
        query += ''' ORDER by returns'''
        data = pd.read_sql(query, con=self.db)
        self.close()
        return data

    def get_trade_loss(self):
        self.connectDB()
        query = '''SELECT algorithm,close_time, sum(case when  status = 'loss' then 1 else 0 end) as loss,
                       sum(case when status = 'profit' then 1 else 0 end) as profit,
                       sum(case when status = 'open' then 1 else 0 end) as open,
                       round(sum(case when  status = 'loss' then 1.0 else 0.0 end)/count(*),2) as loss_ratio,
                       round(sum(case when status = 'profit' then 1.0 else 0.0 end)/count(*),2)  as profit_ratio,
                       round(sum(close*number_of_shares -current*number_of_shares ),2)  as returns
                    FROM Statistics GROUP by close_time having returns <= 0  ORDER by returns'''        
        data = pd.read_sql(query, con=self.db)
        self.close()
        return data
    
    def get_trade_win(self):
        self.connectDB()
        query = '''SELECT algorithm,close_time, sum(case when  status = 'loss' then 1 else 0 end) as loss,
                       sum(case when status = 'profit' then 1 else 0 end) as profit,
                       sum(case when status = 'open' then 1 else 0 end) as open,
                       round(sum(case when  status = 'loss' then 1.0 else 0.0 end)/count(*),2) as loss_ratio,
                       round(sum(case when status = 'profit' then 1.0 else 0.0 end)/count(*),2)  as profit_ratio,
                       round(sum(close*number_of_shares -current*number_of_shares ),2)  as returns
                    FROM Statistics GROUP by close_time having returns > 0  ORDER by returns'''        
        data = pd.read_sql(query, con=self.db)
        self.close()
        return data
    

    def get_trade_close_time(self):
        self.connectDB()
        query = '''SELECT close_time, sum(case when  status = 'loss' then 1 else 0 end) as loss,
                       sum(case when status = 'profit' then 1 else 0 end) as profit,
                       sum(case when status = 'open' then 1 else 0 end) as open,
                       round(sum(case when  status = 'loss' then 1.0 else 0.0 end)/count(*),2) as loss_ratio,
                       round(sum(case when status = 'profit' then 1.0 else 0.0 end)/count(*),2)  as profit_ratio,
                       round(sum(close*number_of_shares -current*number_of_shares ),2)  as returns
                    FROM Statistics GROUP by close_time ORDER by returns'''        
        data = pd.read_sql(query, con=self.db)
        self.close()
        return data
    
    def get_trade_algorithm(self):
        self.connectDB()
        query = '''SELECT algorithm,sum(case when  status = 'loss' then 1 else 0 end) as loss,
                       sum(case when status = 'profit' then 1 else 0 end) as profit,
                       sum(case when status = 'open' then 1 else 0 end) as open,
                       round(sum(case when  status = 'loss' then 1.0 else 0.0 end)/count(*),2) as loss_ratio,
                       round(sum(case when status = 'profit' then 1.0 else 0.0 end)/count(*),2)  as profit_ratio,
                       round(sum(close*number_of_shares -current*number_of_shares ),2)  as returns
                    FROM Statistics GROUP by algorithm ORDER by profit_ratio desc'''        
        data = pd.read_sql(query, con=self.db)
        self.close()
        return data
    
    def get_trade_symbol(self):
        self.connectDB()
        query = '''SELECT symbol,type,sum(case when  status = 'loss' then 1 else 0 end) as loss,
                       sum(case when status = 'profit' then 1 else 0 end) as profit,
                       sum(case when status = 'open' then 1 else 0 end) as open,
                       round(sum(case when  status = 'loss' then 1.0 else 0.0 end)/count(*),2) as loss_ratio,
                       round(sum(case when status = 'profit' then 1.0 else 0.0 end)/count(*),2)  as profit_ratio,
                       round(sum(close*number_of_shares -current*number_of_shares ),2)  as returns
                    FROM Statistics GROUP by symbol,type ORDER by profit_ratio desc'''        
        data = pd.read_sql(query, con=self.db)
        self.close()
        return data


    def close(self):
        self.db.close()

    def deleteDB(self,db):
        if os.path.exists(db):
            os.remove(db)
        else:
            print("The file does not exist")

In [32]:
def showTable(data,index=False):
    print(tabulate(data, headers='keys', tablefmt='psql',showindex=index))

In [33]:
index = PricePattern.BullishEngulfing.value - 1
db = db_list[index]
data_directory = data_directory_list[index]
pattern=pattern_list[index]

In [34]:
trade_data = pd.DataFrame(columns = ['algorithm', 'loss', 'profit','open','loss_ratio','profit_ratio'])
manager = StatisticsManager(hammer_db)
for db in db_list:
    manager.name = db
    row_data = manager.get_trade_algorithm()
    trade_data = trade_data.append(row_data, ignore_index=True)
trade_data
showTable(trade_data)

+---------------------------+--------+----------+--------+--------------+----------------+-----------+
| algorithm                 |   loss |   profit |   open |   loss_ratio |   profit_ratio |   returns |
|---------------------------+--------+----------+--------+--------------+----------------+-----------|
| BullishHammer             |    308 |      293 |      0 |         0.51 |           0.49 |  20504    |
| BullishHarami             |    316 |      286 |      0 |         0.52 |           0.48 |  17296    |
| BullishInverseHammer      |    303 |      275 |      0 |         0.52 |           0.48 |  -1939.55 |
| BullishMorningStar        |    175 |      112 |      0 |         0.61 |           0.39 | -10389.8  |
| BullishPiercingLine       |     97 |       46 |      0 |         0.68 |           0.32 |  -7637.35 |
| BullishThreeWhiteSoldiers |     95 |       89 |      0 |         0.52 |           0.48 |  -2940.7  |
| BullishEngulfing          |    313 |      253 |      0 |         0.55 |

In [57]:
trade_list = [hammer_db,           
           bullish_harami_db,
           inverse_hammer_db,         
           bullish_engulfing_db
          ]
pattern_list = ['hammer',           
           'bullish_harami',
           'inverse_hammer',         
           'bullish_engulfing']
manager = StatisticsManager(hammer_db)

days_map = {"hammer":[],"bullish_harami":[],"inverse_hammer":[],"bullish_engulfing":[]};
index = 0
for db in db_list:
    manager.name = db
    data = manager.get_trade_loss()
    data['close_time'] = data['close_time'].apply(pd.to_datetime).dt.date.astype(str)
    if  data['algorithm'].values[0] == 'BullishHammer' :         
        days_map["hammer"] = data['close_time'].unique().tolist()  
        
    if  data['algorithm'].values[0] == 'BullishHarami' :         
        days_map["bullish_harami"]= data['close_time'].unique().tolist()  
        
    if  data['algorithm'].values[0] == 'BullishInverseHammer' :         
        days_map["inverse_hammer"]= data['close_time'].unique().tolist() 
        
    if  data['algorithm'].values[0] == 'BullishEngulfing' :         
        days_map["bullish_engulfing"]=data['close_time'].unique().tolist()  
    index +=1


showTable(data)

+------------------+--------------+--------+----------+--------+--------------+----------------+-----------+
| algorithm        | close_time   |   loss |   profit |   open |   loss_ratio |   profit_ratio |   returns |
|------------------+--------------+--------+----------+--------+--------------+----------------+-----------|
| BullishEngulfing | 2021-05-12   |     12 |        4 |      0 |         0.75 |           0.25 |  -2838.05 |
| BullishEngulfing | 2021-05-16   |     12 |        4 |      0 |         0.75 |           0.25 |  -2188.55 |
| BullishEngulfing | 2021-05-01   |     10 |        6 |      0 |         0.63 |           0.38 |  -2014.2  |
| BullishEngulfing | 2021-05-07   |     11 |        5 |      0 |         0.69 |           0.31 |  -1835.8  |
| BullishEngulfing | 2021-05-02   |     10 |        6 |      0 |         0.63 |           0.38 |  -1739.9  |
| BullishEngulfing | 2021-04-28   |      8 |        8 |      0 |         0.5  |           0.5  |  -1064    |
| BullishEngulfing 

In [58]:
days_map

{'hammer': ['2021-05-12',
  '2021-05-02',
  '2021-05-23',
  '2021-05-07',
  '2021-05-20',
  '2021-06-02',
  '2021-04-28',
  '2021-05-01',
  '2021-05-29',
  '2021-05-14',
  '2021-04-27',
  '2021-05-06',
  '2021-05-16',
  '2021-05-27',
  '2021-05-17'],
 'bullish_harami': ['2021-05-12',
  '2021-05-02',
  '2021-05-14',
  '2021-05-07',
  '2021-05-27',
  '2021-05-17',
  '2021-05-23',
  '2021-04-27',
  '2021-05-29',
  '2021-06-02',
  '2021-04-30',
  '2021-05-20',
  '2021-04-28',
  '2021-05-06',
  '2021-05-01',
  '2021-05-03',
  '2021-06-01',
  '2021-06-03',
  '2021-05-16'],
 'inverse_hammer': ['2021-05-07',
  '2021-05-12',
  '2021-05-02',
  '2021-05-14',
  '2021-04-27',
  '2021-06-02',
  '2021-05-01',
  '2021-05-23',
  '2021-06-01',
  '2021-05-28',
  '2021-05-20',
  '2021-05-16',
  '2021-05-27',
  '2021-05-29',
  '2021-05-17',
  '2021-05-03',
  '2021-04-30',
  '2021-04-29',
  '2021-05-04'],
 'bullish_engulfing': ['2021-05-12',
  '2021-05-16',
  '2021-05-01',
  '2021-05-07',
  '2021-05-02',
  

In [66]:
day = set(days_map['hammer']).intersection(days_map['bullish_harami']).intersection(days_map['bullish_engulfing']).intersection(days_map['inverse_hammer'])
day

{'2021-04-27',
 '2021-05-01',
 '2021-05-02',
 '2021-05-07',
 '2021-05-12',
 '2021-05-14',
 '2021-05-16',
 '2021-05-17',
 '2021-05-23',
 '2021-05-27',
 '2021-05-29',
 '2021-06-02'}

In [65]:
type(day)
len(day)

12

In [29]:
data['close_time']

0     2021-05-12
1     2021-05-16
2     2021-05-01
3     2021-05-07
4     2021-05-02
5     2021-04-28
6     2021-06-02
7     2021-05-17
8     2021-05-29
9     2021-05-27
10    2021-04-27
11    2021-05-14
12    2021-06-03
13    2021-05-03
14    2021-05-23
15    2021-05-06
16    2021-06-01
17    2021-05-22
Name: close_time, dtype: object